# CSPB 3104 Assignment 10:
## Instructions

> This assignment is to be completed and uploaded to 
moodle as a python3 notebook. 

> Submission deadlines are posted on moodle. 

> The questions  provided  below will ask you to either write code or 
write answers in the form of markdown.

> Markdown syntax guide is here: [click here](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet)

> Using markdown you can typeset formulae using latex.

> This way you can write nice readable answers with formulae like thus:

>> The algorithm runs in time $\Theta\left(n^{2.1\log_2(\log_2( n \log^*(n)))}\right)$, 
wherein $\log^*(n)$ is the inverse _Ackerman_ function.

__Double click anywhere on this box to find out how your instructor typeset it. Press Shift+Enter to go back.__


## Question 1: Arbitrage Opportunities

An arbitrage in finance is a way to make money for sure without much risk. For instance, suppose 1 USD yields 5 Chinese RMB and 1 RMB trades for 0.4 Euros, and 1 Euro trades for 0.52 USD,  you have an arbitrage opportunity
wherein you can take x USD, obtain 5x RMB, and in turn trade them for 2x Euros and finally end up with 1.02x USD back. Every time you exercise this opportunity, you get a 2 cent profit (assuming you can trade for free and there is no spread in the buying and selling prices, but that is another can of worms).

In theory, arbitrage cannot exist in efficient markets but they can persist for a short amount of time. Electronic traders can then search for such opportunities in real time and rapidly exploit them.

You are given a basket of $n$ currencies and for each pair of currencies $(C_i,C_j)$, you are given a ratio
$R_{i,j}$ that says that one unit of currency $C_i$ is currently fetching $R_{i,j}$ units of $C_j$.

__(A)__ Given the ratios $R_{i,j}$ write an efficient algorithm that can detect if any arbitrage opportunities currently exist. What is the running time of your algorithm?

(__Hint:__ Make a graph out of the problem data. What does the presence of arbitrage mean for this graph? )

__(B)__ We are not just interested in finding an arbitrage opportunity, but also interested in finding the shortest length opportunity, defined as one that involves the smallest number of trades. Naturally, given that prices change
and also given that there are trading fees, such opportunities are more desirable. 

Write an algorithm to find the smallest length arbitrage opportunity. What is the running time for your algorithm?

(__Hint:__ Modify the Floyd Warshall algorithm you learned this week. In particular, how do you detect if there is a negative weight cycle of length k in the graph?)




### Answer 1 (Expected Length: 12 lines).

__(A)__ 
Making a graph out of the problem data means that for the arbitrage opportunities, a directed graph should be made where $C_i$ has an edge with $C_j$ with an edge weight is $R_{i,j}$. It is also worth mentioning that there can be an edge from $C_j$ to $C_i$, with an edge weight of $\frac{1}{R_{i,j}}$\
There is an arbitrage opportunity in the presence of a negative cycle. There is a gain when there is a higher value when circling back to $C_i$ from a node that has an edge to it. For example, when circling back to $USD$ in the example, there was a 2 cent gain; there was an opportunity. As the edge weights cannot be multiplications and the ratios are multiplications, we need to use the logarithm of each ratio. Given that some ratios are less than 1, there are negative edge weights, and given that vertices are arranged in a cycle, there is a negative cycle. To detect a negative cycle, we can run the Bellman-Ford algorithm. This would look like the following:
1) Start with USD = 0, this is the source node. Set every other node's distance to $\infty$.
2) Set the weight of every edge to $log(R_{i,j})$
3) Relax all the edges $V - 1$ times, where $V$ is the number of vertices. Each relaxation updates the node values.
4) If the value of USD increases from 0, going from the edge that finalises the cycle, there is an arbitrage opportunity.
If the value of USD can be constantly updated, this means there is a negative cycle in the graph.\
Take the example from this problem:
- Let the edge weights be the logarithm of each ratio: 
    - $w(USD, RMB) = log(5) = 0.698$
    - $w(RMB, EUR) = log(0.4) = -0.397$
    - $w(EUR, USD) = log(0.52) = -0.28$
- Running the Bellman-Ford algorithm given these edge weights:
    - USD = 0
    - RMB = 0.698
    - EUR = 0.301
- Relaxing further, we get:
    - USD = 0.02
    - RMB = 0.7
    - EUR = 0.303

Note: the relax operation is different in this case, as the value of the source node gets updated with a greater value, instead of a lesser one. This means there is a gain of 2 cents in this cycle. As the value of USD can increase, there is a negative cycle. Thus, there is an arbitrage opportunity.\
The running time of this algorithm is the same as the Bellman-Ford algorithm, meaning it is of $\Theta(|V||E|)$, where $V$ is the number of vertices and $E$ is the number of edges.
    

__(B)__ 
As an arbitrage opportunity is represented by a cycle in the graph. We need to find the shortest cycle in the graph.\
For this, we can use a modified version of the Floyd Warshall algorithm. This would look like the following:\
Convert the given exchange ratios $R_{i,j}$ to their logarithmic values $w(i,j) = -\log(R_{i,j})$ (to handle multiplication as addition).

1) Initialise a distance matrix of V * V, where $V$ is the number of vertices. They should have the following characteristics:
    - $D[i][i] = 0$ This means the distance from each currency back to itself is 0.
    - $D[i][j] = w(i,j)$ This is the weight of every edge from vertex $i$ to vertex $j$. Remember, these weighs are in the form $log(R_{i,j})$.
    - If there is no edge between vertices, then $D[i][j] = \infty$
    
2) Given the $D$ matrix, iterate through it and consider all paths that follow (paths of length $k$, where $k$ is from 1 to n)
3) Update $D$ by relaxing the distances given the edge weights by choosing the min of these two values:
    - $D[i][j] = min(D[i][j], D[i][k] + D[k][j])$
4) Once the iteration has been completed, the matrix now the "all pairs shortest path", meaning the shortest path from one vertex to another.
5) Detect if there is any negative weight cycle. As seen in the past implementation, this means there is an arbitrage opportunity. This means checking if $D[i][i] < 0".
6) Find the minimum of these values to obtain the smallest length arbitrage opportunity. This means there is an opportunity for arbitrage.

The running time of this algorithm is the same as Floyd-Warshall's, with a running time of $\Theta(V^3)$, where $V$ is the number of vertices.



----

### Question 2: Longest Paths.

Given a weighted, directed graph $G$, the longest simple path from vertex $s$ to $t$ is a path that (a) goes from $s$ to $t$, (b) cannot revisit a vertex along the path, and (c) has the maximum weight among all the paths going from $s$ to $t$.

__(A)__ Show using an example that for any graph $G$, that the longest path problem is not equivalent to solving the shortest path problem by negating the edge weights. (*Hint* Negating edge weights will work but for a common problem that makes shortest paths undefined.)

__(B)__ Show that if the graph $G$ is a DAG, the longest path problem can be solved by negating the edge weights and solving a shortest path problem. What is the running time cost?


### Answer 2 (Expected Length: 6 lines)

__(A)__
Negating edge weighs doesn't always work to find the longest path of a given graph $G$. Consider an example graph that contains positive weight cycles. This graph $G$ contains the following characteristics:
- Vertices: {A, B, C, D}

- Edges: (they have been negated)
    - $(A, B) -> w(A, B) = -2$
    - $(B, D) -> w(B, D) = -1$
    - $(C, B) -> w(C, B) = -3$
    - $(D, C) -> w(C, D) = -4$

- Running the Bellmand-Ford algorithm, from $(A,B), (B,D), (D,C), (C,B)$ yields the following results in the first iteration (considering A as the source node): 
    - A, $d = 0$
    - B, $d = -10$
    - C, $d = -7$
    - D, $d = -3$
- All edges have been relaxed after this first iteration, now for the second iteration:
    - A, $d = 0$
    - B, $d = -18$
    - C, $d = -15$
    - D, $d = -11$
- The third iteration would look like the following:
    - A, $d = 0$
    - B, $d = -26$
    - C, $d = -23$
    - D, $d = -19$

The graph can be relaxed further after the third iteration, meaning there is a negative weight cycle. This means that the paths to destination nodes is undefined. Therefore, negating the edge weights does not always work for finding the longest path.

__(B)__

If the given graph $G$ is acyclic, then negating the edge weights yields the longest path from the source node to each node. There are no negative weight cycles that cause the paths to destination nodes to go undefined. Given that there are no cycles, we can use the Bellman-Ford algorithm to find the longest path by negating the edge weights and finding the shortest paths. We can run this algorithm with negative values and there is no risk that any shortest path is undefined, as there are no cycles (negated positive weight cycles).

The running time of this algorithm is the same as the discussed Bellman-Ford algorithm. It has a time complexity of $\Theta(|V||E|)$


----

### Question 3: Being In the Right Place At the Right Time.

You are helping a secret agent plan a series of rendezvous in a capital. The rendezvous are happening along different stations of a subway line at precisely hours $1, 2, \ldots, n$.  There are $m$ subway stations, each with an ID numbered $1, \ldots, m$.
The rendezvous at hour $j$ occurs at station $S_j$:

$$\begin{array}{|l|c|c|c|c|c|}
\hline
\text{Rendezvous} & 1 & 2 & \cdots & n-1 & n \\
\hline
\text{Station ID} & S_1 & S_2 & \cdots & S_{n-1} & S_n \\
\hline \end{array}$$

For instance, The rendezvous at hour 1 might happen at station 5, the rendezvous at hour 2 might happen at station 12, and the rendezvous at hour 3 might happen at station 1.  In this case, $S_1 = 5, S_2 = 12,$ and $S_3 = 1$.  __Note:__ Your answer should be for any sequence of rendezvous, not for this specific example.

The spy starts at station $1$ at time $0$ and further, must at all costs attend rendezvous $n$. In each hour, the agent is limited to travelling no more than $K$ stations along the line.  Thus, she must decide whether to skip some of the rendezvous in favor of others.

Provide an algorithm to calculate the maximum number of rendezvous the agent can make with the constraint that she must make rendezvous $n$. What is the running time?

__Hint:__ As usual, it comes down to defining a graph and solving a suitable problem.


### Answer 3 (expected length: 6 lines)

For this problem, we need to define a graph given the information. 
- We know that the agent must be at $S_j$ at hour $j$ for the rendezvous to happen.
- The agent is limited to travelling no more than $K$ stations along the line. This means that she can't go to a station that is more than $K$ stations away.
- The agent needs to be at $S_n$ at hour $n$.
- We want to find the longest path from station 1, $S_0 = 1$ to $S_n$.
- The station index $S_j$ is where the rendezvous at hour $j$ takes place.
The algorithm for finding the maximum number of rendezvous would look like the following:
1. Create a DAG based on which stations the agent can travel to. (Consider the constraint $K$, as she cannot travel farther than this).
2. Topologically sort the DAG based on the rendezvous hours so as to obtain the traversal in order. This topological sort means the agent will be visiting the stations in chronological order.
3. Initialise an array, let's call it $D$. This array identifies the maximum number of rendezvous that the agent can make, given each node. For the starting node, $D[1,0] = 0$. This is the maximum number of rendezvous the agent can make when she's at station $1$ (that's why the first index is $1$), at time $0$ (given the second index of the array). This is initially set to $0$, given that the agent has not been in any rendezvous yet. We also set every other value in the array (node and time) to $-\infty$, because we are looking for the maximum number of nodes to traverse.
4. Traverse each node (in topological order so as to respect the chronological order) and for each node, update the array at station, with the amount of incoming edges. We need to consider the constraint, $K$ as to not go too far on the graph. The max number of rendezvous at $S_j, t$ is determined by looking at the incoming edges and choosing the one that offers the maximum amount of rendezvous. Iterating through all nodes means we get to $S_n$, where we can obtain the maximum number of rendezvous given that we have to make it that station at hour $n$.

The running time for this approach to the problem is of $\Theta(|V|^3)$, where $V$ is the number of vertices or stations. As this is a dynamic programming solution, it may be slower but nonetheless effective.

----

## Question 4: U-turn if you want to.

A $N \times N$ maze is defined by a graph with vertices $(i,j)$ for $1 \leq i \leq n$ and $1 \leq j \leq n$.
Each node $(i,j)$ is connected to a subset of its four adjacent neighbors $\{(i+1, j), (i, j+1), (i-1, j), (i, j-1) \}$.

The grid is laid out so that $(1,1)$ is the south west corner and $(n,n)$ is the northeast corner.

The vehicle starts at $(1,1)$ *oriented north* and needs to reach $(n,n)$ *oriented east*.

It can travel along the four cardinal directions $N, E, W, S$ and rapidly change these direction by 
making a left or right turn. Eg., changing heading from $N$ to $E$ 
requires making a right turn, $N$ to $W$ requires a left turn and $N$ to $S$
requires either 2 left or 2 right turns.

Find the minimimum cost path from $(1,1)$ oriented north to $(n,n)$ oriented east, 
where the cost is defined as the number of 
edges plus $2 \times$ the number of left turns plus $3 \times$ the number of right turns.

What is the running time of your algorithm?

*Hint* Define a new graph that helps us track not just the vehicle location but also its current travel direction.
Define edge weights appropriately so that the shortest cost problem on this new graph will solve the original problem.

### Answer 4 (Expected Length: 8 lines)

Given our $N * N$ maze, we need to find the shortest path from $(1, 1)$ to $(n, n)$. For this approach we will be using a graph, to determine the state of the vehicle (its travel direction and location). The algorithm would look like the following:

1. Create a new graph, the vertices are valid positions in the maze. Each vertex has the form $(i, j)$.
    - Each vertex would contain the four cardinal points (expressed as nodes as well), with edge weights like the following:
        - 1 for going forward
        - 2 for going to the left of the node
        - 3 for going to the right
        - 4 or 6 for U-turning (this depends on wether two lefts or two rights would be valid).
    - Each vertex would be connected to its adjacent neighbours via its cardinal points. A node to the left would be connected via the left for example.

2. Initialise a 2D array to identify the minimum cost to reach $(i, j)$. Let's call this array $D$. Set the cost $D[1][1] = 0$ because this is our starting point. The others would be set to infinity or a very large number. $D[n][n]$ represents the minimum cost for the vehicle to get to $(n, n)$.
3. Initialise another 2D array that keeps track of the nodes that are traversed to find the shortest path.
4. Run Dijkstra's Shortest Path Algorithm to find the minimum path from $(1, 1)$ to $(n, n)$. Our array keeps track of the nodes that need to be traversed to obtained the shortest path at each step, so the this array will get updated. 
5. Recover the solution from our array by revealing the shortest path to get to $(n, n)$.
The solution recovery for this algorithm is similar to our dynamic programming walkthrough, as it contains a record of the shortest path to get to each node. 

The running time of our algorithm would be of $\Theta(N^4)$. This algorithm is rather slow, taking into account the creation of the graph ($\Theta(N^2)$), running Dijkstra's Shortest Path Algorithm on each node, and recovering the solution.


----

## Testing your solutions -- Do not edit code beyond this point